In [ ]:
import cv2
import numpy as np
from skimage import exposure

def load_and_preprocess_image(image_path, target_size=None):
    # Load image with error handling
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Error loading image from {image_path}")

    # Resize if target size is specified
    if target_size:
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_LANCZOS4)

    # Advanced preprocessing
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(img_lab)

    # CLAHE enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)

    # Merge channels
    enhanced_lab = cv2.merge([l, a, b])
    enhanced_bgr = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    # Denoise
    denoised = cv2.fastNlMeansDenoisingColored(enhanced_bgr)

    return denoised, img


In [ ]:
def apply_advanced_filters(image):
    # Bilateral Filter for edge-preserving smoothing
    bilateral = cv2.bilateralFilter(image, 9, 75, 75)

    # Guided Filter
    guided = cv2.ximgproc.guidedFilter(
        guide=image,
        src=image,
        radius=3,
        eps=100
    )

    # Non-local Means Denoising
    nlm = cv2.fastNlMeansDenoisingColored(image)

    # Anisotropic Diffusion
    def anisotropic_diffusion(img, niter=20, kappa=50, gamma=0.1, step=1):
        img = img.astype('float32')
        for _ in range(niter):
            diff = np.zeros_like(img)
            for d in [(1,0),(-1,0),(0,1),(0,-1)]:
                shifted = np.roll(np.roll(img, d[0], 0), d[1], 1)
                gradient = shifted - img
                weight = np.exp(-(gradient/kappa)**2)
                diff += weight * gradient
            img += gamma * diff
        return np.clip(img, 0, 255).astype('uint8')

    aniso = anisotropic_diffusion(image)

    return bilateral, guided, nlm, aniso

In [ ]:
def advanced_edge_detection(image):
    # Canny with automatic threshold detection
    sigma = 0.33
    median = np.median(image)
    lower = int(max(0, (1.0 - sigma) * median))
    upper = int(min(255, (1.0 + sigma) * median))
    edges_canny = cv2.Canny(image, lower, upper)

    # Multi-scale Sobel
    scales = [1, 2, 4]
    sobel_multi = np.zeros_like(image, dtype=np.float64)

    for scale in scales:
        sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=2*scale+1)
        sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=2*scale+1)
        magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
        sobel_multi += magnitude

    sobel_multi = cv2.normalize(sobel_multi, None, 0, 255, cv2.NORM_MINMAX)

    # Scharr operator
    scharr_x = cv2.Scharr(image, cv2.CV_64F, 1, 0)
    scharr_y = cv2.Scharr(image, cv2.CV_64F, 0, 1)
    scharr = np.sqrt(scharr_x**2 + scharr_y**2)

    return edges_canny, sobel_multi.astype(np.uint8), scharr

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def create_advanced_model(input_shape, num_classes):
    # Use EfficientNet as base model
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Fine-tuning setup
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        BatchNormalization(),
        Dense(512, activation='swish'),
        Dropout(0.5),
        BatchNormalization(),
        Dense(256, activation='swish'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    # Advanced training setup
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
    )

    return model

# Advanced training configuration
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

# Data augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomContrast(0.2)
])

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras.mixed_precision import set_global_policy

# Enable mixed precision training
set_global_policy('mixed_float16')

def create_advanced_transfer_model(input_shape, num_classes):
    # Use EfficientNetV2 as base model
    base_model = EfficientNetV2L(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Advanced feature extraction
    inputs = tf.keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='swish')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    # Compile with advanced metrics
    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=1e-4,
            weight_decay=1e-5
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            'accuracy',
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    return model